In [9]:
import numpy as np
import pandas as pd
from larcv import larcv
import math

In [2]:
finalstate_leptons = [13, -13, 11, -11]
protons = [2212, -2212]
cpis = [211, -211]
npis = [111]
gammas = [22]
eles = [11, -11]

In [6]:
def get_n_voxels(dim, event_clusters):
    
    n_voxels = 0
    
    if dim == '3d':
        clusters = event_clusters.as_vector().front().as_vector()
        for cluster in clusters:
            n_voxels += cluster.as_vector().size()
            
    if dim == '2d':
        for plane in range(0, 3):
            clusters = event_clusters.sparse_cluster(plane)
            clusters = clusters.as_vector()
            for cluster in clusters:
                n_voxels += cluster.as_vector().size()
            
    return n_voxels

def get_tot_dep_e(dim, event_clusters):
    
    tot_dep_e = 0
    
    if dim == '3d':
        
        clusters = event_clusters.as_vector().front().as_vector()
        for cluster in clusters:
            for voxel in cluster.as_vector():
                tot_dep_e += voxel.value()
                
        return tot_dep_e
    
    if dim == '2d':
    
        for plane in range(0, 3):
            
            clusters = event_clusters.sparse_cluster(plane)
            clusters = clusters.as_vector()
            for cluster in clusters:
                for voxel in cluster.as_vector():
                    tot_dep_e += voxel.value()
                    
        return tot_dep_e

def get_dep_energy(dim, event_clusters, particle_idx):
    
    if dim == '3d':
        
        clusters = event_clusters.as_vector().front().as_vector()
        cluster = clusters[particle_idx]
        
        sum_charge = 0
        for voxel in cluster.as_vector():
            sum_charge += voxel.value()
        
        return sum_charge
        
    if dim == '2d':
    
        for plane in range(0, 3):
            
            clusters = event_clusters.sparse_cluster(plane)
            clusters = clusters.as_vector()
            cluster = clusters[particle_idx]
        
            sum_charge = 0
            for voxel in cluster.as_vector():
                sum_charge += voxel.value()
                
        return sum_charge
    
    return -1
                

def to_dataframe(filename, dim = '3d'):
    
    df = pd.DataFrame(columns=['pot', 'nu_e', 'nu_vtx_x', 'nu_vtx_y', 'nu_vtx_z',
                               'n_voxels',
                               'true_neutrinoid', 'pred_neutrinoid', 'pred_nue', 'pred_numu', 'pred_nc',
                               'true_prot', 'pred_prot', 'pred_prot0', 'pred_prot1', 'pred_prot2',
                               'true_cpi', 'pred_cpi', 'pred_cpino', 'pred_cpiyes',
                               'true_npi', 'pred_npi', 'pred_npino', 'pred_npiyes',
                               'lep_pdg', 'lep_e', 'lep_costheta', 'lep_thetayz', 'lep_dep_e', 'em_dep_e', 'tot_dep_e',
                               'lead_prot_p', 'lead_prot_costheta',
                               'lead_cpi_p', 'lead_cpi_costheta',
                               'lead_npi_p', 'lead_npi_costheta'])
    
    io = larcv.IOManager()
    io.add_in_file(filename)
    io.initialize()

    for i in range(io.get_n_entries()):
    
        if i % 1000 == 0:
            print("On entry ", i, " of ", io.get_n_entries())
        
        io.read_entry(i)
        
        #
        # Save kinematics
        #
        
        pot = -9999
        nu_e = -9999
        nu_vtx_x = -9999.
        nu_vtx_y = -9999.
        nu_vtx_z = -9999.
        n_voxels = -9999
        lep_pdg = -9999
        lep_e = -9999
        lep_costheta = -9999
        lep_thetayz = -9999
        lep_dep_e = -9999
        em_dep_e = 0
        tot_dep_e = -9999
        lead_prot_p = -9999
        lead_prot_costheta = -9999
        lead_cpi_p = -9999
        lead_cpi_costheta = -9999
        lead_npi_p = -9999
        lead_npi_costheta = -9999
        
        event_cluster3d = larcv.EventSparseCluster3D.to_sparse_cluster(io.get_data("cluster3d", "sbndsegmerged"))
        event_cluster2d = larcv.EventSparseCluster2D.to_sparse_cluster(io.get_data("cluster2d", "sbndsegmerged"))
        
        mc_truth = larcv.EventParticle.to_particle(io.get_data("particle", "sbndsegmerged"))
        
        all_protons_p = []
        all_protons_costheta = []
        
        all_cpis_p = []
        all_cpis_costheta = []
        
        all_npis_p = []
        all_npis_costheta = []
        
        if dim == '3d':
            cluster3d = larcv.EventSparseCluster3D.to_sparse_cluster(io.get_data("cluster3d", "sbndneutrino"))
            cls = cluster3d.as_vector().front().as_vector()
            meta = cluster3d.as_vector().front().meta()
            assert(len(cls) == 1)
            nu_vtx_id = cls[0].as_vector()[0].id()
            nu_vtx = meta.coordinates(nu_vtx_id)
            nu_vtx_x = nu_vtx[0]
            nu_vtx_y = nu_vtx[1] 
            nu_vtx_z = nu_vtx[2]
        
        if dim == '3d': n_voxels = get_n_voxels('3d', event_cluster3d)
        if dim == '2d': n_voxels = get_n_voxels('2d', event_cluster2d)
        
        if dim == '3d': tot_dep_e = get_tot_dep_e('3d', event_cluster3d)
        if dim == '2d': tot_dep_e = get_tot_dep_e('2d', event_cluster2d)
            
        for j, mct in enumerate(mc_truth.as_vector()):
                        
            # Lepton
            if mct.pdg_code() in finalstate_leptons:
                lep_pdg = mct.pdg_code()
                lep_e = mct.energy_init()
                try:
                    lep_costheta = mct.pz() / mct.p()
                    lep_thetayz = math.atan(mct.py() / mct.pz()) / 3.1415 * 180.
                except:
                    pass
                
                if dim == '3d': lep_dep_e = get_dep_energy('3d', event_cluster3d, j)
                if dim == '2d': lep_dep_e = get_dep_energy('2d', event_cluster2d, j)
                    
            # Proton
            if mct.pdg_code() in protons:
                all_protons_p.append(mct.p())
                try:
                    all_protons_costheta.append(mct.pz() / mct.p())
                except:
                    pass
                
            # C pi
            if mct.pdg_code() in cpis:
                all_cpis_p.append(mct.p())
                try:
                    all_cpis_costheta.append(mct.pz() / mct.p())
                except:
                    pass
                
            # N pi
            if mct.pdg_code() in npis:
                all_npis_p.append(mct.p())
                try:
                    all_npis_costheta.append(mct.pz() / mct.p())
                except:
                    pass
                
            # EM: 
            # If it is a gamma or a non primary electron,
            # save the deposited energy as electromagnetic one.
            if mct.pdg_code() in gammas or (mct.pdg_code() in eles and mct.track_id() != 1) :
                if dim == '3d': em_dep_e += get_dep_energy('3d', event_cluster3d, j)
                if dim == '2d': em_dep_e += get_dep_energy('2d', event_cluster2d, j)
                
                
        # Leading proton
        try:
            index_leading_prot = all_protons_p.index(max(all_protons_p))
            lead_prot_p = all_protons_p[index_leading_prot]
            lead_prot_costheta = all_protons_costheta[index_leading_prot]
        except:
            pass
        
        # Leading cpi
        try:
            index_leading_cpi = all_cpis_p.index(max(all_cpis_p))
            lead_cpi_p = all_cpis_p[index_leading_cpi]
            lead_cpi_costheta = all_cpis_costheta[index_leading_cpi]
        except:
            pass
        
        # Leading npi
        try:
            index_leading_npi = all_npis_p.index(max(all_npis_p))
            lead_npi_p = all_npis_p[index_leading_npi]
            lead_npi_costheta = all_npis_costheta[index_leading_npi]
        except:
            pass
         
            
        #
        # Save predicted info
        #
    
        neut_id = larcv.EventTensor1D.to_tensor(io.get_data("tensor1d", "label_neut"))
        neut_id = larcv.as_ndarray(neut_id.as_vector().front())
        pred_neutrinoid =  np.argmax(neut_id)
        
        prot = larcv.EventTensor1D.to_tensor(io.get_data("tensor1d", "label_prot"))
        prot = larcv.as_ndarray(prot.as_vector().front())
        pred_prot =  np.argmax(prot)
        
        cpi = larcv.EventTensor1D.to_tensor(io.get_data("tensor1d", "label_cpi"))
        cpi = larcv.as_ndarray(cpi.as_vector().front())
        pred_cpi =  np.argmax(cpi)
        
        npi = larcv.EventTensor1D.to_tensor(io.get_data("tensor1d", "label_npi"))
        npi = larcv.as_ndarray(npi.as_vector().front())
        pred_npi =  np.argmax(npi)
        
        
        #
        # Save true info
        #
    
        label_neut = larcv.EventParticle.to_particle(io.get_data("particle", "neutID"))
        label_neut = label_neut.as_vector().front().pdg_code()
        
        label_prot = larcv.EventParticle.to_particle(io.get_data("particle", "protID"))
        label_prot = label_prot.as_vector().front().pdg_code()
        
        label_cpi = larcv.EventParticle.to_particle(io.get_data("particle", "cpiID"))
        label_cpi = label_cpi.as_vector().front().pdg_code()
        
        label_npi = larcv.EventParticle.to_particle(io.get_data("particle", "npiID"))
        label_npi = label_npi.as_vector().front().pdg_code()

        
        # pot is per event, by truth information:
        if label_neut == 0:
            # nueCC
            pot = 1.99e16
        elif label_neut == 1:
            # numuCC
            pot = 1.83e14
        else:
            # NC
            pot = 5.19e14
        
        df.loc[i] = [pot, nu_e, nu_vtx_x, nu_vtx_y, nu_vtx_z,
                     n_voxels,
                     label_neut, pred_neutrinoid, neut_id[0], neut_id[1], neut_id[2],
                     label_prot, pred_prot, prot[0], prot[1], prot[2],
                     label_cpi, pred_cpi, cpi[0], cpi[1],
                     label_npi, pred_npi, npi[0], npi[1],
                     lep_pdg, lep_e, lep_costheta, lep_thetayz, lep_dep_e, em_dep_e, tot_dep_e,
                     lead_prot_p, lead_prot_costheta,
                     lead_cpi_p, lead_cpi_costheta,
                     lead_npi_p, lead_npi_costheta]
        
    return df

In [7]:
filename = '/Users/mdeltutt/Downloads/out_sbnd_3d_n5_r6_mb64_bpl2_nf32_lrstriangle_clr_classic_i1000.h5'
df_3d = to_dataframe(filename, dim='3d')
df_3d.to_pickle(filename + '.pkl')

On entry  0  of  7306
On entry  1000  of  7306
On entry  2000  of  7306
On entry  3000  of  7306
On entry  4000  of  7306
On entry  5000  of  7306
On entry  6000  of  7306
On entry  7000  of  7306


In [8]:
filename = '/Users/mdeltutt/Downloads/out_sbnd_2d_n5_r6_mb64_bpl2_nf32_lrstriangle_clr_classic_i600.h5'
df_2d = to_dataframe(filename, dim='2d')
df_2d.to_pickle(filename + '.pkl')

On entry  0  of  7348
On entry  1000  of  7348
On entry  2000  of  7348
On entry  3000  of  7348
On entry  4000  of  7348
On entry  5000  of  7348
On entry  6000  of  7348
On entry  7000  of  7348
